In [1]:
import pandas as pd
import pickle 
from tqdm import tqdm
import torch

In [5]:
with open('../data/chembl/05_chembl_dataset_pyg.pkl', 'rb') as f:
    chembl = pickle.load(f)

In [6]:
len(chembl)

714906

In [8]:
chembl[0]

PairData(edge_index_p=[2, 82], edge_index_d=[2, 82], x_p=[36, 71], x_d=[36, 71], edge_attr_p=[82, 6], edge_attr_d=[82, 6], charge_prot=2, charge_deprot=1, num_nodes=36, reference_value=12.204999923706055, internal_id=[2], smiles_prop='Nc1cc[n+](Cc2cccc(-c3cccc(C[n+]4ccc(N)c5ccccc54)c3)c2)c2ccccc12', smiles_deprop='[NH-]c1cc[n+](Cc2cccc(-c3cccc(C[n+]4ccc(N)c5ccccc54)c3)c2)c2ccccc12', chembl_id='CHEMBL6354', reaction_center='20')

In [50]:
prot_smiles = []
deprot_smiles = []
prot_charges = []
deprot_charges = []
pka = []
chembl_ids = []
centers = []

for data in tqdm(chembl):
    prot_smiles.append(data.smiles_prop)
    deprot_smiles.append(data.smiles_deprop)
    prot_charges.append(data.charge_prot)
    deprot_charges.append(data.charge_deprot)
    pka.append(data.reference_value.item())
    chembl_ids.append(data.chembl_id)
    centers.append(data.reaction_center)


100%|██████████| 714906/714906 [00:10<00:00, 65431.38it/s]


In [51]:
len(chembl_ids)

714906

In [52]:
chembl_df = pd.DataFrame({'chembl_id': chembl_ids,
                          'pka': pka,
                          'center': centers,
                          'prot_smiles': prot_smiles,
                          'deprot_smiles': deprot_smiles,
                          'prot_charge': prot_charges,
                          'deprot_charge': deprot_charges})

In [53]:
chembl_df.head()

,chembl_id,pka,center,prot_smiles,deprot_smiles,prot_charge,deprot_charge
0,CHEMBL6354,12.205,20,Nc1cc[n+](Cc2cccc(-c3cccc(C[n+]4ccc(N)c5ccccc5...,[NH-]c1cc[n+](Cc2cccc(-c3cccc(C[n+]4ccc(N)c5cc...,2,1
1,CHEMBL6354,12.807,21,[NH-]c1cc[n+](Cc2cccc(-c3cccc(C[n+]4ccc(N)c5cc...,[NH-]c1cc[n+](Cc2cccc(-c3cccc(C[n+]4ccc([NH-])...,1,0
2,CHEMBL429017,12.205,16,Nc1cc[n+](Cc2ccc(CCc3ccc(C[n+]4ccc(N)c5ccccc54...,[NH-]c1cc[n+](Cc2ccc(CCc3ccc(C[n+]4ccc(N)c5ccc...,2,1
3,CHEMBL429017,12.807,17,[NH-]c1cc[n+](Cc2ccc(CCc3ccc(C[n+]4ccc(N)c5ccc...,[NH-]c1cc[n+](Cc2ccc(CCc3ccc(C[n+]4ccc([NH-])c...,1,0
4,CHEMBL6361,12.205,20,Nc1cc[n+](Cc2ccc(/C=C\c3ccc(C[n+]4ccc(N)c5cccc...,[NH-]c1cc[n+](Cc2ccc(/C=C\c3ccc(C[n+]4ccc(N)c5...,2,1


In [5]:
with open('../data/chembl/04_chembl_dataset_pyg.pkl', 'rb') as f:
    raw_chembl = pickle.load(f)

In [45]:
unstack_mols = []
seen = set()
c = 0
p = []
for id_ in tqdm(chembl_df['chembl_id']):
    if id_ not in seen:
        seen.add(id_)
        mols = raw_chembl[id_]['mols']
        c += len(mols)
        for mol in mols:
            unstack_mols.append([mol[0], mol[1]])
            p.append(mol[0].GetProp('pKa_number'))

100%|██████████| 714906/714906 [00:06<00:00, 112830.07it/s]


In [47]:
chembl_df['prot_mol'] = [i[0] for i in unstack_mols]

In [48]:
chembl_df['deprot_mol'] = [i[1] for i in unstack_mols]

In [28]:
chembl_df['acid_basic'] = p 

In [33]:
torch.save(chembl_df, '/scratch/dz1061/pkasolver-data/chembl_processed.pt')

In [5]:
chembl_df.shape

(714906, 10)

## End.